# WSDOT Initial Data Observations

This notebook explores the basic relationships and overall quality of the tolling data provided by WSDOT.

In [1]:
suppressWarnings(suppressMessages(library(tidyverse)))
suppressMessages(library(lubridate))
library(RSQLCipher)

In [2]:
Sys.setenv("SQL_KEY"=Sys.getenv("HOT_KEY"))

In [3]:
db_path = "../../../data/hot.db"

# compute fraction of missing data in a column
frac.na = function(x) { mean(is.na(x)) }
replace_null = function(x) {
    na_if(na_if(na_if(x, "11183834060272721597"), "-8355759756528748941"), "8974271441158017554")
}

# import tables
trips = load_table(db_path, "trips") # ~16M rows
bos = load_table(db_path, "bos") # ~10M rows
bad_bos = load_table(db_path, "bad_bos") # 36.4K rows
census = load_table(db_path, "census") # ~331K rows, 331K unique acct/plate pairs
bad_census = load_table(db_path, "bad_census") # ~46K rows

In [4]:
make_2way = function(df, var1, var2) {
    table(eval(substitute(var1), df), eval(substitute(var2), df), 
         dnn=c(substitute(var1), substitute(var2)), useNA="ifany") %>% prop.table %>% round(3)
}

---
## Table overview

### `trips`
All of the trip data are present, except for ~22% of the `is_hov` and `tag_status` entries.  A quick check of these rows shows that they (nearly) all paid a fare.  Presumably these can therefore be reclassified as `is_hov=0`.

In [110]:
trips %>%
    head(50000) %>%
    execute %>%
    summarize_all(frac.na) %>% t

trips %>%
    #filter(is.null(is_hov)) %>%
    head(50000) %>%
    execute %>%
    make_2way(is_hov, fare==0)

trip_id,0.00000
def_id,0.00000
txn_time,0.00000
fare,0.00000
entry_time,0.00000
is_hov,0.21842
exit_time,0.00000
tag_status,0.21842
pay_type_code,0.00000
filter_type,0.00000
entry_plaza_id,0.00000


      fare == 0
is_hov FALSE  TRUE
  0    0.453 0.003
  1    0.002 0.324
  <NA> 0.217 0.001

Approximately 33% of trips are HOV trips.

In [91]:
trips %>%
    head(50000) %>%
    execute %>%
    mutate(is_hov = replace_na(is_hov, 0)) %>%
    summarize(mean(is_hov, na.rm=T))

"mean(is_hov, na.rm = T)"
<dbl>
0.32556


### `bos`

We are missing around 10% of tag IDs, 6% of "secondary" plates, 4% of accounts, and 2% of plate states.  95% of licence plate types are missing, which suggests that the default (no particular type) is coded as `NULL`. We are also missing 0.6% of ZIP codes, and nearly 90% of ZIP+4 codes.

In [144]:
bos.d = bos %>%
    head(10000) %>%
    execute %>%
    mutate(plate_state_sec = na_if(plate_state_sec, -8355759756528748941))

t(summarize_all(bos.d, frac.na))

#sort(table(bos.d$plate_state_pri, useNA="ifany"), decreasing=T)[1:10]

txn_id,0.0000
trip_id,0.0000
veh_class,0.0000
tag_id,0.0496
posted_account,0.0353
plate_state_pri,0.0000
plate_state_sec,0.0680
plate_state,0.0125
lic_plate_type_code,0.9502
zip_code,0.0069
plus4_code,0.8767


There are approximately 1.6 million unique account/plate pairs, out of the 10 million or so rows in the table.

In [18]:
# takes a while to run!
bos %>%
    select(posted_account, plate_state_sec) %>%
    distinct %>%
    summarize(count=n()) %>%
    execute

count
<dbl>
1644590


### `bad_bos`

Out of the 36,000 rows in this table, there are only 3,000 unique account/plate pairs.  This represents 0.2% each of trips and accounts.  It is probably not worth it to try to deduplicate this table and append to the existing BOS table.

In [ ]:
# 3016 unique acct/plate pairs
bad_bos %>%
    select(posted_account, plate_state_sec) %>%
    distinct %>%
    summarize(count=n()) %>%
    execute

### `census`

This table has no missing data.

In [23]:
# ~331K rows, 331K unique acct/plate pairs
census %>%
    head(1000) %>%
    execute %>%
    summarize_all(frac.na) %>% t

id,0
is_plate,0
city,0
state,0
zip_code,0
is_exact,0
fips,0
county,0
cty_subdivision,0
block,0


### `bad_census`

The vast majority of these 46,000 duplicate census records do not have a geocode match.  Only 5,000 do.  Given this small number, it probably does not make sense to try to deduplicate these records and append them to the existing census file.

In [25]:
bad_census %>%
    head(1000) %>%
    execute %>%
    summarize_all(frac.na) %>% t

bad_census %>%
    filter(matchfound == "Match") %>%
    summarize(count=n()) %>%
    execute

id,0.000
account,0.000
plate,1.000
city,0.000
state,0.000
zip_code,0.000
matchfound,0.000
exactness,0.922
STATE.1,0.922
county,0.922
cty_subdivision,0.922


count
<dbl>
5503


***
## Joining Trip, BOS, and Census Block Data

### Trip-BOS join
Joining trip to BOS data is relatively quick, once an index is built for `trip_id` in the BOS table.  We notice immediately, however, that over 40% of the trips are unable to be matched to an account or a plate.

In [45]:
trips_bos_q = left_join(
    select(trips, -filter_type, -tag_status, -txn_time, -entry_lane_num, -exit_lane_num),
    select(bos, trip_id, tag_id, acct=posted_account, plate=plate_state_sec, plate_pri=plate_state_pri, plate_state, zip_code), 
    by="trip_id") %>%
    head(50000)

trips_bos = execute(trips_bos_q, col_types="iidiliiiicccccc") %>%
    mutate(is_hov = replace_na(is_hov, 0),
           plate = replace_null(plate))
t(summarise_all(trips_bos, frac.na))

trip_id,0.00000
def_id,0.00000
fare,0.00000
entry_time,0.00000
is_hov,0.00000
exit_time,0.00000
pay_type_code,0.00000
entry_plaza_id,0.00000
exit_plaza_id,0.00000
tag_id,0.56156
acct,0.49646


Digging into the missing data more, we observe the following

- Whenever we have an account, we also have a plate.  But sometimes we have a plate without an account.
- About half of trips have an account, and about 57% have a plate.

In [42]:
make_2way(trips_bos, is.na(acct), is.na(plate))
make_2way(trips_bos, is.na(acct), is.na(zip_code))
make_2way(trips_bos, is.na(plate), is.na(zip_code))

           is.na(plate)
is.na(acct) FALSE  TRUE
      FALSE 0.465 0.039
      TRUE  0.063 0.434

           is.na(zip_code)
is.na(acct) FALSE  TRUE
      FALSE 0.501 0.003
      TRUE  0.062 0.435

            is.na(zip_code)
is.na(plate) FALSE  TRUE
       FALSE 0.519 0.008
       TRUE  0.043 0.429

Around 66% of the missing accounts, and 76% of the missing plates are HOV trips. 
**No HOV trips have accounts, plates, or tag information.**

In [10]:
make_2way(trips_bos, is.na(acct), is_hov)
make_2way(trips_bos, is.na(plate), is_hov)
make_2way(trips_bos, is.na(tag_id), is_hov)

           is_hov
is.na(acct)     0     1
      FALSE 0.504 0.000
      TRUE  0.171 0.326

            is_hov
is.na(plate)     0     1
       FALSE 0.571 0.000
       TRUE  0.103 0.326

             is_hov
is.na(tag_id)     0     1
        FALSE 0.438 0.000
        TRUE  0.236 0.326

98% of license plates are from Washington.  They do not account for the missing non-HOV accounts.

In [11]:
sum(trips_bos$plate_state != "WA", na.rm=T) 
cat("\n")
table(trips_bos$plate_state) %>% prop.table %>% round(4)
cat("\n\n")
make_2way(trips_bos, is.na(acct) & !is_hov, plate_state=="WA")

[1] 509


    AK     AZ     BC     CA     CO     DE     FL     GA     HI     IA     ID 
0.0002 0.0000 0.0008 0.0051 0.0003 0.0000 0.0004 0.0000 0.0001 0.0002 0.0007 
    IL     IN     KS     MA     MD     MI     MN     MT     NC     NV     NY 
0.0002 0.0001 0.0000 0.0000 0.0000 0.0001 0.0001 0.0002 0.0000 0.0001 0.0001 
    OH     OR     PA     SC     TX     UT     VA     VT     WA     WY 
0.0000 0.0082 0.0001 0.0000 0.0004 0.0002 0.0001 0.0001 0.9820 0.0001 

                     plate_state == "WA"
is.na(acct) & !is_hov FALSE  TRUE  <NA>
                FALSE 0.007 0.492 0.330
                TRUE  0.004 0.064 0.103

### Trip-Census join
After joining BOS and census data to the trips table, nearly 60% of the trips are unable to be matched to a census block group.

In [16]:
trips_bos_census = left_join(
    mutate(trips_bos_q, id=coalesce(acct, plate, plate_pri)),
    select(census, id, fips),
    by="id") %>% 
execute(col_types="iidiliiiicccccccc")
    
t(summarise_all(trips_bos_census, frac.na))

# assume NA is_hov are FALSE.
trips_bos_census = mutate(trips_bos_census, is_hov = replace_na(is_hov, 0))

trip_id,0.00000
def_id,0.00000
fare,0.00000
entry_time,0.00000
is_hov,0.21842
exit_time,0.00000
pay_type_code,0.00000
entry_plaza_id,0.00000
exit_plaza_id,0.00000
tag_id,0.56156
acct,0.49646


Among non-HOV trips, only 61% of trips can be matched to a census block group.
Among _domestic_ non-HOV trips, **only 73% of trips can be matched to a census block group.**

In [17]:
make_2way(trips_bos_census, is.na(acct) & is.na(plate), is.na(fips))
make_2way(trips_bos_census, is.na(zip_code), is.na(fips))
make_2way(trips_bos_census, !is.na(acct) & !is.na(zip_code), is.na(fips))
make_2way(trips_bos_census, !is.na(fips), !is_hov)
make_2way(trips_bos_census, !is.na(fips), !is_hov & plate_state=="WA")

                          is.na(fips)
is.na(acct) & is.na(plate) FALSE  TRUE
                     FALSE 0.413 0.158
                     TRUE  0.000 0.429

               is.na(fips)
is.na(zip_code) FALSE  TRUE
          FALSE 0.413 0.150
          TRUE  0.000 0.437

                               is.na(fips)
!is.na(acct) & !is.na(zip_code) FALSE  TRUE
                          FALSE 0.028 0.471
                          TRUE  0.385 0.116

            !is_hov
!is.na(fips) FALSE  TRUE
       FALSE 0.326 0.261
       TRUE  0.000 0.413

            !is_hov & plate_state == "WA"
!is.na(fips) FALSE  TRUE  <NA>
       FALSE 0.333 0.149 0.105
       TRUE  0.003 0.407 0.003

All of the matched census records are in-state.

In [136]:
table(trips_bos_census$state)


   WA 
19242 

In [39]:
make_2way(trips_bos_census, is.na(fips) & !is_hov, plate_state=="WA")
make_2way(trips_bos_census, is.na(trip_id), plate_state=="WA")

                     plate_state == "WA"
is.na(fips) & !is_hov FALSE  TRUE  <NA>
                FALSE 0.003 0.407 0.328
                TRUE  0.007 0.149 0.105

              plate_state == "WA"
is.na(trip_id) FALSE  TRUE  <NA>
         FALSE 0.010 0.556 0.433

Around 74% of users can be matched to census data.

In [27]:
users_bos_census = left_join(
    mutate(trips_bos_q, id=coalesce(acct, plate, plate_pri)),
    select(census, id, state, fips),
    by="id") %>% 
    select(id, fips) %>%
    group_by(id) %>%
    summarize(fips=mean(!is.na(fips)), count=n()) %>%
execute

In [28]:
mean(users_bos_census$fips)

[1] 0.7385412

73.2% of users who cannot be matched to a census record used the system only once, compared to 70.9% of users who could be matched.  This difference is significant at the 1% level.

In [35]:
make_2way(users_bos_census, fips, count>1)
with(users_bos_census, chisq.test(fips, count>1))

    count > 1
fips FALSE  TRUE
   0 0.191 0.070
   1 0.523 0.215


	Pearson's Chi-squared test with Yates' continuity correction

data:  fips and count > 1
X-squared = 10.301, df = 1, p-value = 0.00133
